In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2

In [5]:
import matplotlib.pylab as plt
import matplotlib as mpl
plt.rcParams['figure.figsize'] = (12.0, 6.0)
#%matplotlib notebook
%matplotlib inline
%load_ext base16_mplrc
%base16_mplrc light default

The base16_mplrc extension is already loaded. To reload it, use:
  %reload_ext base16_mplrc
Setting plotting theme to default-light. Palette available in b16_colors


In [9]:
import cPickle as pickle
from itertools import permutations, combinations
from fuzzywuzzy import fuzz, process
from project_modules.flavorNetworkBuilder.recipe_weights import compute_all_weights, match_ingredients, compute_single_avg_weight

In [256]:
from collections import deque

## testing recommender

In [14]:
def get_range(dictionary, begin, end):
  return dict(itertools.islice(dictionary.iteritems(), begin, end+1)) 

In [26]:
ls ../../data

backbone_edges_dict.pkl   full_edges_dict.pkl
doc_joined.pkl            recipes.pkl
doc_split.pkl             second_ing_flav_dict.pkl
drink_links.pkl           superseded_data/
from_paper/


In [10]:
data_dir = '../../data/'

In [31]:
file_name = 'ldc_recipes.pkl'
with open(data_dir+file_name, 'r') as f:
    ldc_recipes  = pickle.load(f)
    
file_name = 'nyt_recipes.pkl'
with open(data_dir+file_name, 'r') as f:
    nyt_recipes  = pickle.load(f)

In [50]:
all_recipes = {}
for cocktail in ldc_recipes:
    all_recipes.update(cocktail)
all_recipes.update(nyt_recipes)

In [63]:
file_name = 'second_ing_flav_dict.pkl'
with open(data_dir+file_name, 'r') as f:
    comp_ing_dict = pickle.load(f)

In [68]:
file_name = 'backbone_edges_dict.pkl'
with open(data_dir+file_name) as f:
    edge_dict = pickle.load(f)

In [69]:
file_name = 'full_edges_dict.pkl'
with open(data_dir+file_name) as f:
    full_edge_dict = pickle.load(f)

In [141]:
file_name = 'match_dict.pkl'
with open(data_dir+file_name) as f:
    match_dict = pickle.load(f)

In [18]:
with open('./avg_recipe_weights.pkl', 'r') as f:
    avg_w = pickle.load(f)
with open('./new_avg_recipe_weights.pkl', 'r') as f:
    new_avg_w = pickle.load(f)

In [19]:
with open('./random_weights_new.pkl', 'r') as f:
    random_weights = pickle.load(f)

In [20]:
with open('./new_avg_recipe_weights.pkl', 'r') as f:
    avg_recipe = pickle.load(f)
    
with open('./new_avg_random_weights.pkl', 'r') as f:
    avg_random = pickle.load(f)

In [52]:
cocktail_ing_lists = all_recipes.values()
cocktail_set = set([i.strip().lower() for sublist in cocktail_ing_lists for i in sublist if i])

In [62]:
len(cocktail_set)

1442

In [39]:
match_dict = {k:match_ingredients([k], comp_ing_dict) for k in unique_ing_list}

In [ ]:
%prun {k:match_ingredients([k], comp_ing_dict) for k in unique_ing_list}

In [70]:
match_ingredients(['whiskey'], comp_ing_dict)

[u'whiskey',
 u'the basic fractions of whiskey',
 u'malt whiskey',
 u'canadian whiskey',
 u'japanese whiskey']

In [59]:
with open('./match_dict.pkl', 'w')  as f:
    pickle.dump(match_dict, f)

In [96]:
small_match_dict  = {k:match_ingredients([k], comp_ing_dict) for k in ['rye whiskey', 'orange']}

In [180]:
cocktail_set_list

[u'ardbeg whisky',
 u'coco lopez cream of coconut',
 u'amaro montenegro',
 u'grape juice']

In [275]:
%page all_recipes

In [189]:
cocktail_set_list = list(cocktail_set)

In [343]:
filter_list = ['chocolate bitters', u'pineapple', 'pineapple puree', 'apple','ginger beer','peach bitters','mango nectar']

In [344]:
# ingred_opt(all_recipes['Elderflower Martini'])

In [345]:
def ingred_opt(drink_recipe):
    max_score = ('max_ingrd', 0)
    for ingredient in cocktail_set_list:
        score = 0
        pairings = permutations(drink_recipe + [ingredient], 2)
        for pair in pairings:
            try:
                score += compute_single_avg_weight(pair, comp_ing_dict, edge_dict, match_dict)
            except KeyError:
                pass
        if score > max_score[1] and ingredient not in filter_list:
            max_score = (ingredient, score)
    return max_score

In [368]:
new_drink = ['gin', 'cucumber', 'lemon' ]

In [369]:
ingred_opt(new_drink)

(u'cardamom bitters', 216.0)